<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#特征工程-CTR任务，user_id和item_id的数量那么大，也常规embedding吗？" data-toc-modified-id="特征工程-CTR任务，user_id和item_id的数量那么大，也常规embedding吗？-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>特征工程-CTR任务，user_id和item_id的数量那么大，也常规embedding吗？</a></span></li><li><span><a href="#样本优化-都说数据是上限，推荐系统ctr模型中，构造正负样本有哪些实用的trick？" data-toc-modified-id="样本优化-都说数据是上限，推荐系统ctr模型中，构造正负样本有哪些实用的trick？-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>样本优化-都说数据是上限，推荐系统ctr模型中，构造正负样本有哪些实用的trick？</a></span></li><li><span><a href="#模型评估-如何评价CTR预估模型的效果？" data-toc-modified-id="模型评估-如何评价CTR预估模型的效果？-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>模型评估-如何评价CTR预估模型的效果？</a></span></li><li><span><a href="#模型评估-离线AUC涨了，在线CTR没涨可能有哪些原因？" data-toc-modified-id="模型评估-离线AUC涨了，在线CTR没涨可能有哪些原因？-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>模型评估-离线AUC涨了，在线CTR没涨可能有哪些原因？</a></span></li><li><span><a href="#模型评估-特征重要度评估" data-toc-modified-id="模型评估-特征重要度评估-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>模型评估-特征重要度评估</a></span></li></ul></div>

## 特征工程-CTR任务，user_id和item_id的数量那么大，也常规embedding吗？

> **推荐阅读**⭐️⭐️⭐️⭐️⭐️
> - [CTR任务，user_id和item_id的数量那么大，也常规embedding吗？](https://www.zhihu.com/question/515899875/answer/2504796344)

如果是大厂的话，就是直接加。大厂所使用的parameter server一般都有特征的准入与逐出机制。尽管模型代码上都一视同仁地将所有user_id, item_id加入模型，但是只有那些出现超过一定次数的id，才最终被parameter server分配空间。而且对于非活跃用户、过时下架的item，parameter server也会逐出，释放空间。

## 样本优化-都说数据是上限，推荐系统ctr模型中，构造正负样本有哪些实用的trick？

> **推荐阅读**⭐️⭐️⭐️⭐️⭐️
> - [都说数据是上限，推荐系统ctr模型中，构造正负样本有哪些实用的trick？](https://www.zhihu.com/question/324986054/answer/1746373510)

1. 最好的trick就是**保证数据精准前提下，如无必要，不要采样**。既然数据是模型的上限，就不应该破坏这个上限。
2. 按用户活跃度筛选样本。高活用户行为丰富，低活、回归、新用户行为稀疏，同一套特征体系无法对两者都友好。**高活用户样本应该单独建模**，低活用户应该配合用户增长策略、生态建设联动优化。
3. **调权优于采样。调整样本权重产生不同的梯度贡献，本质上和采样的期望效果一致**。但采样对用户的长尾兴趣、item库内长尾类目不友好。没有计算资源压力的前提下不建议采样。
4. focal loss解决hard sample mining。hard sample mining对推荐系统同样重要。如果不知道怎么找准对应场景的困难样本，同时暂时也还在用交叉熵做二分类loss的话，可以无脑迁移focal loss。

## 模型评估-如何评价CTR预估模型的效果？

> **推荐阅读**⭐️⭐️⭐️⭐️⭐️
> - [如何评价CTR预估效果？](https://www.zhihu.com/question/54009615)

1. 评价CTR预估效果来说，假设给了你一个CTR预估模型，logloss和AUC是两个比较常用的评价指标。**logloss更关注和观察数据的吻合程度，AUC更关注rank order**。
2. 此外还有GAUC这类指标，**如果优化目标更关注体量，比如点击量、DAU等，那应该以AUC为准**。
3. copc=实际的点击率/模型预测的点击率，主要衡量model整体预估的偏高和偏低，同样越接近1越好，一般情况下在1附近波动。计算偏差的时候需要考虑下训练模型是是否有采样逻辑，如果有采样逻辑，那么计算copc时需要考虑采样比。
4. 有一种比较常用的方式来评估模型是高估还是低估，采集线上日志，按pctr排序后按指定步长分桶，统计每个分桶的actr和pctr均值，横轴表示pctr，纵轴表示actr，画出关系图，理想状态下应该是一条斜率为1的线。

## 模型评估-离线AUC涨了，在线CTR没涨可能有哪些原因？

> **推荐阅读**⭐️⭐️⭐️⭐️⭐️
> -  https://www.zhihu.com/question/517418281/answer/2355367968
> - [「评估」AUC离线好,上线差?试试GAUC](https://www.xuebawang.net/t/47417)


1. **特征穿越**：使用了和label强相关的特征导致数据泄漏。


2. **线上线下特征不一致**：在离线统一特征抽取框架，样本在线落盘（碰到的最多的问题）。


3. **在线服务有问题**：通常需要确认线上各个服务的监控，比如特征服务、预估服务等是否稳定，常关注的指标：服务可用性、超时、CPU利用率、流控、模型时效性、特征服务稳定性、请求总量、请求成功率、打分分布等。


4. **强偏置特征建模方式是否正确**：对于一些强bias特征，线上线下的使用方式也是特别重要的，如果使用不当往往离线收益很大而线上纹丝不动或者反向。比如pos bias，在推荐领域，一般放到wide&deep模型的wide侧，离线训练时按实际曝光位置来训练，线上serving时置为0，这对于只追求序关系正确（为什么序关系没有影响，可以从原理上推导，欢迎大家一起讨论）的推荐场景是没有问题的。但是放到广告ctr模型中就存在问题，因为广告场景中还需要依赖pctr做出价计算去收取广告主的钱，这就需要保证pctr预估精准度高，而对所有候选adid的pos bias特征都置为0会引起较大的预估gap。一个可行的方案是先将pos bias置为0，计算所有候选adid的[mid_pctr](https://www.zhihu.com/search?q=mid_pctr&search_source=Entity&hybrid_search_source=Entity&hybrid_search_extra={"sourceType"%3A"answer"%2C"sourceId"%3A2355367968})，按这个排序得到候选adid实际曝光的pos，按这个pos再走一遍模型打分，得到准确的[real_pctr](https://www.zhihu.com/search?q=real_pctr&search_source=Entity&hybrid_search_source=Entity&hybrid_search_extra={"sourceType"%3A"answer"%2C"sourceId"%3A2355367968})，基于这个去做最终的出价，这种方案缺点也很明显，需要预估两次，但一般整个预估链路的耗时大都集中在特征抽取框架，模型前向过程耗时相对较小。


5. **特殊的时间节点**：假期、双十一等。

## 模型评估-特征重要度评估

> **推荐阅读**⭐️⭐️⭐️⭐️⭐️
> - [特征重要度分析方法梳理](https://zhuanlan.zhihu.com/p/609635878)

比较靠谱的方式是：模型训练完成后，将需要评估的特征mask成一个默认值（或者对应的特征做batch内shuffle，不改变特征的分布），然后评估模型预测性能的差异。如果指标跌的厉害，就说明这个特征比较重要，反之，这个特征可以丢弃。